In [17]:
import random
import time
from concurrent.futures import ThreadPoolExecutor

# ---------------- MERGE FUNCTION ----------------
def merge(left, right):
    result = []
    i = j = 0

    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    result.extend(left[i:])
    result.extend(right[j:])
    return result


# ---------------- SINGLE-THREADED MERGE SORT ----------------
def merge_sort_single(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = merge_sort_single(arr[:mid])
    right = merge_sort_single(arr[mid:])
    return merge(left, right)


# ---------------- MULTI-THREADED MERGE SORT ----------------
def merge_sort_parallel(arr, depth=0, max_depth=3):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2

    if depth < max_depth:
        with ThreadPoolExecutor(max_workers=2) as executor:
            left_future = executor.submit(merge_sort_parallel, arr[:mid], depth + 1, max_depth)
            right_future = executor.submit(merge_sort_parallel, arr[mid:], depth + 1, max_depth)

            left = left_future.result()
            right = right_future.result()
    else:
        left = merge_sort_parallel(arr[:mid], depth, max_depth)
        right = merge_sort_parallel(arr[mid:], depth, max_depth)

    return merge(left, right)


# ---------------- MAIN PROGRAM ----------------
if __name__ == "__main__":
    n = 200000    # dataset size
    max_depth = 3  # controls number of threads (2^max_depth)

    arr = [random.randint(0, 100000) for _ in range(n)]
    arr_copy = arr.copy()

    print(f"Array Size: {n}")
    print(f"Max Thread Depth: {max_depth} → Threads used = {2**max_depth}")

    # -------- SINGLE THREAD TEST --------
    start = time.time()
    sorted_single = merge_sort_single(arr)
    end = time.time()
    single_time = end - start
    print(f"\nSingle-thread Merge Sort Time: {single_time:.4f} seconds")

    # -------- MULTI THREAD TEST --------
    start = time.time()
    sorted_multi = merge_sort_parallel(arr_copy, 0, max_depth)
    end = time.time()
    multi_time = end - start
    print(f"Multi-thread Merge Sort Time: {multi_time:.4f} seconds")

    # -------- SPEEDUP --------
    print(f"\nSpeedup = {single_time / multi_time:.2f}x")


Array Size: 200000
Max Thread Depth: 3 → Threads used = 8

Single-thread Merge Sort Time: 0.7743 seconds
Multi-thread Merge Sort Time: 2.0003 seconds

Speedup = 0.39x
